# Experiment Tracking
## Preprocessing

In [71]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
from hyperopt import tpe, hp, Trials, STATUS_OK, fmin
from hyperopt.pyll import scope
import mlflow
import pickle

In [40]:
pd.options.mode.copy_on_write = True

In [41]:
def preprocessing(df):
    # compute target variables: trip duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # keep only trips of less than an hour
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

    # extract trip date components
    df['trip_month'] = pd.Categorical(df.lpep_pickup_datetime.dt.month)
    df['trip_dom'] = pd.Categorical(df.lpep_pickup_datetime.dt.day)
    df['trip_hour'] = pd.Categorical(df.lpep_pickup_datetime.dt.hour)

    # concatenate pickup and dropoff locations
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)

    return df

In [42]:
def one_hot_encoding(df, numerical, categorical, dv=None):
    df[categorical] = df[categorical].astype(str)
    df_dicts = df[categorical + numerical].to_dict(orient='records')

    if not dv:
        dv = DictVectorizer()
        dv.fit(df_dicts)

    X_train = dv.transform(df_dicts)

    return X_train, dv

In [43]:
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)

In [44]:
# read im the raw data
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [45]:
# data cleaning
df_train = preprocessing(df_train)
df_val = preprocessing(df_val)

In [46]:
# variable selection
categorical = ['PU_DO']
numerical = ['trip_distance']

In [47]:
# categorical feature encoding
X_train, dv = one_hot_encoding(df_train, numerical, categorical)
X_val, _ = one_hot_encoding(df_val, numerical, categorical, dv=dv)

In [48]:
target = 'duration'
y_train = df_train[target]
y_val = df_val[target]

In [ ]:
with open('models/preprocessor.b', "wb") as f_out:
    pickle.dump(dv, f_out)

## Manual Logging

In [49]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748987505722, experiment_id='1', last_update_time=1748987505722, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [50]:
mlflow.sklearn.autolog(disable=True)
mlflow.xgboost.autolog(disable=True)

### Lasso

In [51]:
alpha = .01
lr = Lasso(alpha=alpha)
train_model(lr, X_train, y_train)
y_preds = lr.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [52]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.log_param("alpha", alpha)
    mlflow.sklearn.log_model(lr, "model")
    mlflow.log_metric("rmse", rmse)

2025/06/04 00:29:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Xgboost

In [53]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [54]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50,
        verbose_eval=200
    )

    y_preds = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_preds)

    with mlflow.start_run():
        mlflow.set_tag("developer", "Armand Winant")

        mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
        mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
        mlflow.log_params(params)
        mlflow.xgboost.log_model(booster, "model")
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [55]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [56]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:7.66055                           
[200]	validation-rmse:6.34188                         
[257]	validation-rmse:6.34329                         
  0%|          | 0/50 [00:11<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:29:39] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 00:29:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:11.20316                                                    
[200]	validation-rmse:6.33953                                                   
[400]	validation-rmse:6.32469                                                   
[429]	validation-rmse:6.32467                                                   
  2%|▏         | 1/50 [01:40<17:27, 21.37s/trial, best loss: 6.3432850901531985]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:31:13] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 00:31:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:8.70373                                                     
[200]	validation-rmse:6.33591                                                 
[278]	validation-rmse:6.33830                                                 
  4%|▍         | 2/50 [02:38<51:16, 64.10s/trial, best loss: 6.32466791025804]

...

[0]	validation-rmse:9.31912                                                     
[200]	validation-rmse:6.57562                                                   
[249]	validation-rmse:6.57734                                                   
 96%|█████████▌| 48/50 [57:22<01:24, 42.18s/trial, best loss: 6.305945326886427]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:26:52] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 01:26:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:9.95465                                                     
[200]	validation-rmse:6.39741                                                   
[400]	validation-rmse:6.38263                                                   
[469]	validation-rmse:6.38157                                                   
 98%|█████████▊| 49/50 [58:06<00:43, 43.05s/trial, best loss: 6.305945326886427]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:27:36] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 01:27:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 50/50 [58:15<00:00, 69.91s/trial, best loss: 6.305945326886427]


In [60]:
params = {
    'learning_rate': 0.23768760883453516,
    'max_depth': 32,
    'min_child_weight': 1.4255637138891217,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.20333941491253288,
    'reg_lambda': 0.014135076785365224,
    'seed': 42
}

In [69]:
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50,
    verbose_eval=200
)

y_preds = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_preds)

with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.log_params(params)
    mlflow.xgboost.log_model(booster, "model")
    mlflow.log_metric("rmse", rmse)

[0]	validation-rmse:10.35446
[200]	validation-rmse:6.35461
[400]	validation-rmse:6.31678
[581]	validation-rmse:6.30616


2025/06/04 09:13:38 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:13:38] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/04 09:15:45 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/04 09:16:42 WARNING mlflow.models.evaluation.evaluators.shap: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.


### Random Forest

In [ ]:
rf = RandomForestRegressor()
train_model(rf, X_train, y_train)
y_preds = rf.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(rf, "model")
    mlflow.log_metric("rmse", rmse)

### Linear SVR

In [ ]:
svr = LinearSVR(max_iter=10000000)
train_model(svr, X_train, y_train)
y_preds = svr.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(svr, "model")
    mlflow.log_metric("rmse", rmse)

### Gradient Boosting

In [ ]:
gb = GradientBoostingRegressor()
train_model(gb, X_train, y_train)
y_preds = gb.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(gb, "model")
    mlflow.log_metric("rmse", rmse)

### Extra Trees

In [ ]:
et = ExtraTreesRegressor()
train_model(et, X_train, y_train)
y_preds = et.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(et, "model")
    mlflow.log_metric("rmse", rmse)